In [1]:
import optimization.methods as methods

In [13]:
two_stage = methods.two_stage_opt(2, 100, 1.0, 50)

In [14]:
one_stage = methods.non_integer_opt(2, 100, 1.0)

In [15]:
one_stage.corr(two_stage.mean(axis=1))

0.9757660838495961

In [17]:
import data.databaseManager as dbMgr

In [18]:
loop_df = dbMgr.query("""select distinct c.structureid, cp.timeid from contestOwnership co
join contests c on co.contestid = c.contestid
join contestPlayers cp on cp.timeid = c.timeid and cp.playerid = co.playerid and cp.structureid = c.structureid""")

In [102]:
risk_params = [0, .05, .1, .15, .2]

In [103]:
one_stage_ports = {}
one_stage_points = {}
#num_contests = 50
for ix, row in loop_df.iterrows():
    points = dbMgr.query("""select playerid, points from contestPlayers
                        where structureid = {}
                        and timeid = {}""".format(row['structureid'],
                                                  row['timeid'])).set_index('playerid')['points']
    for rp in risk_params:
        port = methods.non_integer_opt(row['structureid'], row['timeid'], rp)
        #mult_port = methods.two_stage_opt(row['structureid'], row['timeid'], rp, num_contests)
        one_stage_points[(row['structureid'], row['timeid'], rp)] = (port * points).sum()
        one_stage_ports[(row['structureid'], row['timeid'], rp)] = port
        #two_stage_ports[(row['structureid'], row['timeid'], rp)] = mult_port

In [61]:
pd.Series(one_stage_points)[2].unstack(level=1).describe()

,0.0,0.2,0.4,0.6,0.8,1.0
count,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000
mean,148.882012,131.594347,128.133158,125.616248,121.960630,118.317521
std,24.016768,9.100693,8.039693,8.049647,7.984107,7.445917
min,95.002361,110.401546,109.129377,107.328644,103.956022,102.006202
25%,132.471250,125.859064,124.390704,121.359068,117.050344,113.561553
50%,146.717729,130.873055,127.988677,125.826893,121.889448,118.346696
75%,161.566635,137.883028,134.021295,132.015039,126.574698,122.634514
max,203.134400,154.855907,148.026214,145.593052,143.068030,139.147195


In [64]:
contest_df = dbMgr.query("""select c.contestid, timeid, stat, value from contestStats st
join contests c on st.contestid = c.contestid
where structureid = 2""")

In [67]:
stat_df = contest_df.pivot_table(index=['contestid','timeid'], columns='stat', values='value')

In [77]:
stat_df['Multiplier'] = stat_df['Top Prize'] / stat_df['Buy In']

In [81]:
filtered_df = stat_df[stat_df['Multiplier'] == 2]

In [86]:
cashlines = filtered_df.groupby(level=1).mean()['Cash Line']

In [92]:
pd.Series(one_stage_points)[2].unstack().join(cashlines, how='inner')

,0.0,0.2,0.4,0.6,0.8,1.0,Cash Line
103,138.158588,110.401546,109.129377,107.328644,103.956022,102.006202,125.88
105,185.266818,136.233049,131.425598,127.665817,121.589553,118.011022,116.59
106,132.742857,119.558716,117.105400,114.774480,111.781988,108.652701,114.08
108,128.983922,131.868268,125.510113,120.592278,115.366786,112.048923,125.80
109,132.430449,126.070684,119.222739,115.788341,110.390568,106.638240,91.00
110,123.473807,118.967200,117.690306,115.715402,112.559466,109.687613,102.21
112,95.002361,120.639887,119.202574,115.811595,111.364706,108.474572,70.98
114,197.951538,127.927249,124.271539,122.403719,119.328187,115.780294,95.24
117,149.536095,125.647444,125.124888,124.179299,122.674525,117.666249,131.68
118,131.112376,114.345443,111.664515,111.263447,110.146967,106.781116,128.40


In [87]:
cashlines.join()

timeid
103    125.88
105    116.59
106    114.08
108    125.80
109     91.00
110    102.21
112     70.98
114     95.24
117    131.68
118    128.40
119    129.12
120    204.82
121    139.94
122    148.56
123    205.08
125    159.80
126    141.20
127    144.44
129    139.84
130    142.12
132    158.80
134    128.66
135    163.58
137    161.22
138    135.02
148    126.52
149    144.80
151    157.62
152    157.54
153    146.72
Name: Cash Line, dtype: float64

In [63]:
contest_df

'select c.contestid, timeid, stat, value from contestStats st\njoin contests c on st.contestid = c.contestid\nwhere structureid = 2'

In [56]:
debug

> /Users/calvin/anaconda3/lib/python3.7/subprocess.py(1576)_try_wait()
   1574             """All callers to this function MUST hold self._waitpid_lock."""
   1575             try:
-> 1576                 (pid, sts) = os.waitpid(self.pid, wait_flags)
   1577             except ChildProcessError:
   1578                 # This happens if SIGCLD is set to be ignored or waiting

ipdb> u
> /Users/calvin/anaconda3/lib/python3.7/subprocess.py(1618)_wait()
   1616                         if self.returncode is not None:
   1617                             break  # Another thread waited.
-> 1618                         (pid, sts) = self._try_wait(0)
   1619                         # Check the pid and loop as waitpid has been known to
   1620                         # return 0 even without WNOHANG in odd situations.

ipdb> u
> /Users/calvin/anaconda3/lib/python3.7/subprocess.py(984)wait()
    982             endtime = _time() + timeout
    983         try:
--> 984             return self._wait(t

ipdb> ix
0
ipdb> q


In [47]:
pd.Series(one_stage_ports)[2].unstack(level=1)

,0,1,2,3,4
86,160.101046,119.715890,112.531429,110.063226,108.842472
87,155.141822,117.249326,110.282162,107.844879,106.627943
88,116.472476,122.641652,113.951404,110.935220,109.444906


In [37]:
points

playerid
3       16.08
4       29.94
5        3.72
6       26.40
7       30.50
9       13.76
12      15.40
18      19.00
22      13.18
25      19.90
26       8.98
30      11.24
32      10.64
38       0.00
39       3.90
41      29.60
42       8.40
46      27.70
47       1.90
54      -0.50
61       1.80
68      15.30
72       6.30
74       0.50
85       3.30
88       8.40
89      22.70
97       9.20
106     29.50
110     25.50
        ...  
1216     2.90
1217     0.20
1218     0.00
1219     0.00
1220     2.30
1221     0.00
1222     3.00
1223     0.00
1224    14.92
1226     5.40
1227    13.20
1228     0.00
1230     5.10
1231     0.00
1232     7.40
1233     5.80
1235     0.00
1236     0.00
1237     0.00
1241     0.00
1242    10.66
1243     1.60
1244     0.70
1245     0.20
1246     0.00
1247     0.00
1248     0.00
1249     3.90
1250     5.40
1251     0.00
Name: points, Length: 420, dtype: float64

In [34]:
import pandas as pd

In [35]:
pd.DataFrame(one_stage_ports)

2                                                                \
            86                                                            87   
             0             1         2         3             4             0   
3     0.000045  8.295280e-02  0.056494  0.046926  4.204733e-02  1.000078e+00   
4     0.000052  4.237753e-02  0.032532  0.028932  2.703199e-02  0.000000e+00   
5     0.000009  2.204729e-02  0.035033  0.039070  4.101971e-02  0.000000e+00   
6     0.000058  5.853029e-02  0.033237  0.022902  1.763003e-02  0.000000e+00   
7     0.000033  2.295192e-02  0.006450  0.000654  3.276155e-24  0.000000e+00   
9     0.000017  2.520200e-02  0.024875  0.025394  2.557354e-02  0.000000e+00   
11    0.000000  6.758261e-03  0.025722  0.031757  3.470353e-02  0.000000e+00   
12    0.000027  1.135154e-02  0.025875  0.030429  3.262899e-02  0.000000e+00   
18    0.000040  2.889844e-02  0.022876  0.020455  1.915413e-02  0.000000e+00   
22    0.000009  4.380204e-02  0.042100  0.041246  4.074162e-02  0.000000e+00   
25    0.000030  3.230007e-02  0.039266  0.041300  4.224573e-02  0.000000e+00   
26    0.000047  7.813088e-03  0.009979  0.010536  1.071300e-02  0.000000e+00   
30    0.000020  1.426230e-02  0.015091  0.015071  1.496517e-02  0.000000e+00   
32         NaN           NaN       NaN       NaN           NaN  0.000000e+00   
41    0.000000  5.315816e-02  0.042509  0.038929  3.714628e-02  0.000000e+00   
42    0.000000  6.047349e-02  0.050524  0.047135  4.544866e-02  0.000000e+00   
46    0.000000  7.239924e-02  0.076051  0.077081  7.760629e-02  0.000000e+00   
54    0.000000  5.715166e-02  0.051263  0.049192  4.816942e-02  0.000000e+00   
61    0.000000  4.950491e-02  0.066782  0.072766  7.577380e-02  0.000000e+00   
63    0.000000  7.046413e-02  0.056322  0.051536  4.915248e-02  0.000000e+00   
68    0.000000  4.419280e-02  0.041199  0.040145  3.957199e-02  0.000000e+00   
73    0.000000  6.302189e-02  0.054145  0.051113  4.961019e-02  0.000000e+00   
84    0.000000  3.210067e-02  0.034539  0.035179  3.551127e-02  0.000000e+00   
85         NaN           NaN       NaN       NaN           NaN           NaN   
88    0.293630  8.200630e-02  0.060038  0.052613  4.891099e-02  9.999843e-01   
89    0.000000  7.080450e-02  0.055105  0.049816  4.718587e-02  0.000000e+00   
97    0.000000  2.970777e-02  0.034343  0.036260  3.723049e-02  0.000000e+00   
106   0.000000  5.395165e-02  0.049119  0.047436  4.660646e-02  0.000000e+00   
110   0.000000  6.360087e-02  0.072775  0.075531  7.692107e-02  0.000000e+00   
156   0.000000  2.145317e-02  0.024556  0.025698  2.630418e-02  2.627916e-06   
...        ...           ...       ...       ...           ...           ...   
1106       NaN           NaN       NaN       NaN           NaN           NaN   
1109  0.000000  4.462421e-02  0.045288  0.045518  4.567867e-02  0.000000e+00   
1113  0.000000  7.504409e-02  0.053314  0.046441  4.301289e-02  0.000000e+00   
1116  0.000000  3.260505e-02  0.048788  0.053946  5.656832e-02  5.000821e-06   
1125  0.000000  3.579488e-02  0.028816  0.026508  2.538470e-02  8.412142e-07   
1144       NaN           NaN       NaN       NaN           NaN           NaN   
1145  0.000000  4.474731e-02  0.050556  0.052258  5.315015e-02  0.000000e+00   
1157       NaN           NaN       NaN       NaN           NaN  5.000821e-06   
1167  0.000000  1.140068e-24  0.006317  0.016239  2.112978e-02  0.000000e+00   
1177  0.000000  7.903240e-24  0.019263  0.026021  2.932310e-02  0.000000e+00   
1178  0.999814  2.269711e-02  0.021886  0.021245  2.082354e-02  0.000000e+00   
1179  0.000000  7.474321e-02  0.054887  0.048201  4.486814e-02  0.000000e+00   
1181       NaN           NaN       NaN       NaN           NaN           NaN   
1182  0.000000  5.461457e-02  0.066443  0.070254  7.216951e-02  0.000000e+00   
1184  0.000000  5.952039e-02  0.073085  0.077480  7.969515e-02           NaN   
1189       NaN           NaN       NaN       NaN           NaN           NaN   
1190  0.00000

In [11]:
import optimization.projections as proj

In [12]:
import optimization.dataPrep as prep

In [13]:
platform = 'fanduel'

In [14]:
timeid = 100

In [15]:
player_cov = rm.get_player_cov(platform, timeid)

In [16]:
ers = proj.get_expected_points(platform, timeid)

In [17]:
data_df = prep.get_platform_data(platform, timeid)

In [18]:
universe = player_cov.index.intersection(ers.index).intersection(data_df.index)

In [19]:
er_uni = ers.reindex(universe)

In [20]:
cov_uni = player_cov.reindex(universe, axis=1).reindex(universe, axis=0)

In [21]:
data_uni = data_df.reindex(universe)

In [22]:
list_constraints = [
    ("bound", "<=", None, 1),
    ("bound", ">=", None, 0),
    ("dot", "==", data_uni['DST'], 1),
    ("dot", "==", data_uni['QB'], 1),
    ("dot", ">=", data_uni['RB'], 2),
    ("dot", "<=", data_uni['RB'], 3),
    ("dot", ">=", data_uni['WR'], 3),
    ("dot", "<=", data_uni['WR'], 4),
    ("dot", ">=", data_uni['TE'], 1),
    ("dot", "<=", data_uni['TE'], 2),
    ("dot", "==", data_uni['Flex'], 7),
    ("dot", "<=", data_uni['salary'], 60000),
]

In [25]:
list_constraints = [
    ("bound", "<=", None, 1),
    ("bound", ">=", None, 0),
    ("dot", "==", 'DST', 1),
    ("dot", "==", 'QB', 1),
    ("dot", ">=", 'RB', 2),
    ("dot", "<=", 'RB', 3),
    ("dot", ">=", 'WR', 3),
    ("dot", "<=", 'WR', 4),
    ("dot", ">=", 'TE', 1),
    ("dot", "<=", 'TE', 2),
    ("dot", "==", 'Flex', 7),
    ("dot", "<=", 'salary', 60000),
]

In [23]:
import pandas as pd

In [28]:
list_constraints = pd.DataFrame(list_constraints)

In [1]:
import data.explorer as exp

In [2]:
exp.structure_search('fanduel', 'HALF', 'all week without kicker')

4

In [3]:
import data.databaseManager as dbMgr

In [5]:
constraint_df = dbMgr.query("""
select type, operator, vec, bound from structureConstraints con
join contestStructure s on con.structureid = s.structureid
where platform = 'fanduel'""")

In [22]:
constraint_df['vec'] = constraint_df['vec'].apply(lambda x: data_uni[x] if x is not None)

0       None
1       None
2         RB
3         TE
4         WR
5     salary
6        DST
7       Flex
8         QB
9         RB
10        TE
11        WR
Name: vec, dtype: object

In [12]:
constraint_list = list(constraint_df.itertuples(index=False, name=None))

In [20]:
constraint_list[0][3] = 5

TypeError: 'tuple' object does not support item assignment

In [8]:
constraint_df

,type,operator,vec,bound
0,bound,<=,None,1.0
1,bound,>=,None,0.0
2,dot,<=,RB,3.0
3,dot,<=,TE,2.0
4,dot,<=,WR,4.0
5,dot,<=,salary,60000.0
6,dot,==,DST,1.0
7,dot,==,Flex,7.0
8,dot,==,QB,1.0
9,dot,>=,RB,2.0


In [14]:
import optimization.methods as opt

In [15]:
opt_var = opt.mean_variance(er_uni, cov_uni, 1, list_constraints)

In [17]:
numcontests = 10

In [18]:
opt.linear_opt(er_uni, list_constraints, False, opt_var*numcontests, numcontests)

,0,1,2,3,4,5,6,7,8,9
1,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,1,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,1,0,0,0


In [94]:
import pulp as pulp

In [95]:
numContests = 10

In [96]:
prob = pulp.LpProblem("FanDuelOpt",pulp.LpMaximize)
choice_dict = {}
for cont in range(numContests):
    choice_dict[cont] = pulp.LpVariable.matrix("Choices"+str(cont),universe,lowBound=0,upBound=1,cat='Integer')
choice_dict['Total'] = pulp.LpVariable.matrix("ChoicesTotal",universe,lowBound=0,upBound=numContests,cat='Integer')
#Y1 = pulp.LpVariable.matrix("Y1",universe,lowBound=0)

In [97]:
import pandas as pd

In [98]:
choice_df = pd.DataFrame(choice_dict)

In [99]:
prob = pulp.LpProblem("FanDuelOpt",pulp.LpMaximize)
#prob += pulp.lpSum(Y1)
prob += pulp.lpDot(choice_dict['Total'], list(er_uni.values))
for cont in range(numContests):
    prob += pulp.lpDot(list(data_uni['salary'].values), choice_dict[cont]) <= 60000
    prob += pulp.lpDot(list(data_uni['DST'].values),choice_dict[cont]) == 1
    prob += pulp.lpDot(list(data_uni['QB'].values),choice_dict[cont]) == 1
    prob += pulp.lpDot(list(data_uni['RB'].values),choice_dict[cont]) >= 2
    prob += pulp.lpDot(list(data_uni['RB'].values),choice_dict[cont]) <= 3
    prob += pulp.lpDot(list(data_uni['WR'].values),choice_dict[cont]) >= 3
    prob += pulp.lpDot(list(data_uni['WR'].values),choice_dict[cont]) <= 4
    prob += pulp.lpDot(list(data_uni['TE'].values),choice_dict[cont]) >= 1
    prob += pulp.lpDot(list(data_uni['TE'].values),choice_dict[cont]) <= 2
    prob += pulp.lpDot(list(data_uni['Flex'].values),choice_dict[cont]) == 7
    #prob += pulp.lpDot(list(filtered_df['FPTS'].values),choice_dict[cont]) >= (crowd * needed_df['FPTS']).sum()
for ix in choice_df.index:
    prob += pulp.lpSum(list(choice_df.iloc[ix].values[:numContests])) == choice_df.iloc[ix]['Total']
prob.solve()

1

In [101]:
choice_df.index = universe

In [103]:
include_exclude = choice_df.applymap(lambda x: bool(x.value())).drop('Total',axis=1)

In [106]:
optimal = include_exclude[include_exclude].dropna(how='all')

In [112]:
data_uni.reindex(optimal.index).sum()

position    WRTEDSTWRQBRBWRRBRB
salary                    60000
points                    113.7
DST                           1
K                             0
QB                            1
RB                            3
TE                            1
WR                            3
Flex                          7
dtype: object

In [114]:
er_uni.reindex(optimal.index).sum()

134.0

In [108]:
optimal.groupby(data_uni['position']).sum()

,0,1,2,3,4,5,6,7,8,9
position,,,,,,,,,,
DST,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
QB,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
RB,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
TE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
WR,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [21]:
opt_var.sum()

9.000000000000004

In [ ]:
prob = pulp.LpProblem("FanDuelOpt",pulp.LpMinimize)
prob += pulp.lpSum(Y1)
for cont in range(numContests):
    prob += pulp.lpDot(list(filtered_df['Cost'].values), choice_dict[cont]) <= 60000
    prob += pulp.lpDot(list(filtered_df['D'].values),choice_dict[cont]) == 1
    prob += pulp.lpDot(list(filtered_df['QB'].values),choice_dict[cont]) == 1
    prob += pulp.lpDot(list(filtered_df['RB'].values),choice_dict[cont]) >= 2
    prob += pulp.lpDot(list(filtered_df['RB'].values),choice_dict[cont]) <= 3
    prob += pulp.lpDot(list(filtered_df['WR'].values),choice_dict[cont]) >= 3
    prob += pulp.lpDot(list(filtered_df['WR'].values),choice_dict[cont]) <= 4
    prob += pulp.lpDot(list(filtered_df['TE'].values),choice_dict[cont]) >= 1
    prob += pulp.lpDot(list(filtered_df['TE'].values),choice_dict[cont]) <= 2
    prob += pulp.lpDot(list(filtered_df['Flex'].values),choice_dict[cont]) == 7
    prob += pulp.lpDot(list(filtered_df['FPTS'].values),choice_dict[cont]) >= (crowd * needed_df['FPTS']).sum()
for ix in choice_df.index:
    prob += pulp.lpSum(list(choice_df.iloc[ix].values[:numContests])) == choice_df.iloc[ix]['Total']
    prob += choice_df.iloc[ix]['Total'] - filtered_df['desired'].iloc[ix] <= Y1[ix]
    prob += choice_df.iloc[ix]['Total'] - filtered_df['desired'].iloc[ix] >= -Y1[ix]
prob.solve()